In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

from wordcloud import WordCloud
import seaborn as sns
import jieba,re

import tensorflow as tf

In [2]:
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split

In [3]:
data_movie = pd.read_csv('../data/movie_info.csv',header=None)

In [4]:
data_movie.head()

,0,1,2,3,4,5,6,7,8
0,“大”人物,2019-01-10,107,北京,2390,柯里昂,2,2019-01-09,说句题外话，看了这么些投机偷懒的类型片，越来越确信如果香港电影能“活”到今天，还真没有韩国电...
1,“大”人物,2019-01-10,107,江苏南京,1114,武侠小王子,4,2019-01-05,包贝尔演坏人真是本色演出，最后被暴打确实让人解气。你们都说翻拍《老手》，但这部真的比《老手》强。
2,“大”人物,2019-01-10,107,北京,622,谢谢你们的鱼,4,2019-01-06,居然比期待的还要好看，可以说是中国目前为止最成功的韩国电影翻拍片了，继承了原作的优点的同时还...
3,“大”人物,2019-01-10,107,辽宁沈阳,442,河狸,4,2019-01-01,看了十分钟就感觉剧情似曾相识，查了下原来是翻拍韩版的《老手》。品质上，比起上一部同样是王千源...
4,“大”人物,2019-01-10,107,北京,208,乌鸦火堂,2,2019-01-09,复刻+洗白，不喜欢。把最不适合带幽默感的地方强行搞笑，显得很儿戏一样，就算照着拍又能咋样，反...


In [5]:
data_movie.columns=['电影名','上映时间','电影时长','点评人地区','点赞数','点评人','评分','点评时间','短评']

In [6]:
# 全部影评Series
movie_comment = data_movie['短评']

In [12]:
movie_comment.shape

(4575,)

In [7]:
from keras.models import Sequential
from keras.layers import Dense, GRU, Embedding, LSTM, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam,RMSprop
from keras.callbacks import EarlyStopping,ModelCheckpoint,TensorBoard,ReduceLROnPlateau

Using TensorFlow backend.


In [8]:
# 预训练词向量
cn_model = KeyedVectors.load_word2vec_format('../data/sgns.zhihu.bigram',binary=False)

In [20]:
# 将词语转化为300维向量
embedding_dim = cn_model['豆瓣'].shape[0]
print('词向量的长度为{}'.format(embedding_dim))
cn_model['豆瓣']

词向量的长度为300


array([  4.09891009e-01,  -7.31108010e-01,  -7.70874977e-01,
         1.09599996e-03,   6.79298997e-01,  -2.03200011e-03,
         3.68887991e-01,  -3.40894997e-01,  -1.20163001e-01,
        -4.75751996e-01,  -3.07318002e-01,   2.50851005e-01,
         1.77943006e-01,  -6.18049026e-01,   8.74464989e-01,
        -2.03355694e+00,  -3.00603986e-01,  -6.92209974e-02,
         3.72557014e-01,   4.68825996e-01,  -1.41500002e-02,
         3.52963001e-01,   6.53542995e-01,  -1.08011998e-01,
        -5.45728981e-01,   9.93412971e-01,  -9.30701017e-01,
        -5.08674979e-01,   6.84489012e-01,  -9.55968022e-01,
        -1.35191500e+00,   2.12259993e-01,   6.61749989e-02,
        -2.50268996e-01,  -8.98180008e-02,   1.37460101e+00,
         2.60491014e-01,  -3.54909986e-01,  -9.05290022e-02,
         2.53457010e-01,  -9.07300040e-03,  -2.01868996e-01,
        -3.02401006e-01,  -3.66710007e-01,   5.07667005e-01,
         5.49599994e-03,   7.87811995e-01,   2.49935001e-01,
        -5.00985026e-01,

In [15]:
# 以预训练模型匹配相似度
cn_model.similarity('会计','出纳')

0.57939309

In [17]:
cn_model.most_similar(positive=['AI'], topn=10)

[('人工智能', 0.7275527119636536),
 ('智能机器人', 0.5681489706039429),
 ('AlphaGo', 0.5666214823722839),
 ('阿法狗', 0.565135657787323),
 ('机器人', 0.5623875856399536),
 ('Master', 0.5579691529273987),
 ('ai', 0.5505872368812561),
 ('alphago', 0.5428926348686218),
 ('图像识别', 0.5237392783164978),
 ('阿尔法', 0.522647500038147)]

In [13]:
cn_model.most_similar(positive=['女人','出轨'], negative=['男人'], topn=1)

[('劈腿', 0.5849198698997498)]

In [9]:
# 将影评分类
positive_info = data_movie[(data_movie.评分>=3)]
negative_info = data_movie[(data_movie.评分<3)]
display(negative_info.head(),positive_info.head())

,电影名,上映时间,电影时长,点评人地区,点赞数,点评人,评分,点评时间,短评
0,“大”人物,2019-01-10,107,北京,2390,柯里昂,2,2019-01-09,说句题外话，看了这么些投机偷懒的类型片，越来越确信如果香港电影能“活”到今天，还真没有韩国电...
4,“大”人物,2019-01-10,107,北京,208,乌鸦火堂,2,2019-01-09,复刻+洗白，不喜欢。把最不适合带幽默感的地方强行搞笑，显得很儿戏一样，就算照着拍又能咋样，反...
5,“大”人物,2019-01-10,107,江苏南京,222,凌睿,2,2019-01-05,表面上是讽刺“大人物”的电影，实质上是教育你别去招惹大人物，也别去和大人物论理，应该乖乖拿上...
6,“大”人物,2019-01-10,107,江苏南京,145,森夏,2,2018-12-26,《老手》的本土化把“本”去掉。有韩国原版兜底还能拍成这个烂样子。能够看出影片挺缺钱的，熟脸都...
9,“大”人物,2019-01-10,107,江苏苏州,107,西楼尘,2,2019-01-06,送钱送包送房，最怕送的是心动；打黑打恶打鬼，最怕打的是人心。可以剃了别人的发，断了别人的腿，...


,电影名,上映时间,电影时长,点评人地区,点赞数,点评人,评分,点评时间,短评
1,“大”人物,2019-01-10,107,江苏南京,1114,武侠小王子,4,2019-01-05,包贝尔演坏人真是本色演出，最后被暴打确实让人解气。你们都说翻拍《老手》，但这部真的比《老手》强。
2,“大”人物,2019-01-10,107,北京,622,谢谢你们的鱼,4,2019-01-06,居然比期待的还要好看，可以说是中国目前为止最成功的韩国电影翻拍片了，继承了原作的优点的同时还...
3,“大”人物,2019-01-10,107,辽宁沈阳,442,河狸,4,2019-01-01,看了十分钟就感觉剧情似曾相识，查了下原来是翻拍韩版的《老手》。品质上，比起上一部同样是王千源...
7,“大”人物,2019-01-10,107,北京,110,朝阳区刘能,3,2018-12-27,无功无过的翻拍，因为不能学韩国黑太过，所以居然往主旋律调子上去靠，也是无语，反倒是王砚辉大叔...
8,“大”人物,2019-01-10,107,北京,83,王大根,3,2019-01-06,在国产翻拍片里寻找那一行“本片改编自某某电影”的小字的难度，已经可以比肩Where’s Wa...


In [10]:
positive_comment = list(positive_info['短评'])
negative_comment = list(negative_info['短评'])

In [11]:
display(len(positive_comment),len(negative_comment))

3812

763

In [27]:
positive_comment

['包贝尔演坏人真是本色演出，最后被暴打确实让人解气。你们都说翻拍《老手》，但这部真的比《老手》强。',
 '居然比期待的还要好看，可以说是中国目前为止最成功的韩国电影翻拍片了，继承了原作的优点的同时还能完成如此完整、成熟并且具有代入感的本土化改编，甚至在现实批判的尺度上一点也不输《老手》。官商勾结、暴力强拆、官僚讽刺和富人丑态，就没有在大陆的大银幕上看到如此敢拍的商业动作片，最可贵的是表演层面几乎全线炸裂，包贝尔的表现好得不可思议，连王千源都略显逊色！',
 '看了十分钟就感觉剧情似曾相识，查了下原来是翻拍韩版的《老手》。品质上，比起上一部同样是王千源主演的翻拍片《破局》要好很多。包贝尔的表演让人眼前一亮，比印象中好很多很多，值得表扬。',
 '无功无过的翻拍，因为不能学韩国黑太过，所以居然往主旋律调子上去靠，也是无语，反倒是王砚辉大叔贡献了最多笑点。原版刘亚仁的反派是让人又爱又恨，跟黄政民大叔一起搭戏完全是锦上添花，个人魅力一点不输；包贝尔演的反派就只是让人恨而已，观众投射的情感是单一的，比如我的内心OS就只是希望王千源赶紧揍他…我觉得这并不是说他演的多好，只是说明他平时观众缘真的太差了。当然，跟他以前的装疯卖傻比倒也算是进步了吧…',
 '在国产翻拍片里寻找那一行“本片改编自某某电影”的小字的难度，已经可以比肩Where’s Wally 了。最出彩的还是王砚辉，竟然还有人说包贝尔演技大爆发？他不就是更加淋漓尽致地发挥出了自己恶心人的天赋吗？本片最大加分点：王千源穿贴身纯色T恤太hot了，头小肩宽背挺胸结实，线条超性感，不愧是中国第一悍匪帅！！（我封的',
 '没看过韩版，超出预期。王千源和一干老戏骨发挥稳定，包贝尔“本色出演”一个招人厌的富二代，倒也挺生动形象的。影片整体风格偏幽默，动作戏、追车戏的剪辑也挺不错。',
 '《老手》抄的香港警匪片，本片抄的《老手》，连动作和桥段都一模一样。我不站赵家，也不站国家，只为中国警匪片的进步点个赞。那无关上头的“口子”开多大，而在于拿捏节奏、塑造人物和技术部门等商业片的核心要素的进步。王千源和包贝尔都找对了戏路。最后商业中心擒贼的一段放在中国也还是有效，至少能打动有钱买票进场的阶层。要说缺点那就是颜色也调得跟韩片似的，鲜艳过分了。',
 '3.5 虽然本土化做的一般，但整体看下来竟然也不差了。聊的社会上中下层问题，某种意

In [12]:
train_tokens_positive = []
for text in positive_comment:
    text = re.sub("[\s+\.\!\/_$%^*(+]+|[+——！，。；：《》【】“”？、~@#￥%……&*（）]+", "",text)
    cut_text = jieba.cut(text)
    cut_list = [i for i in cut_text]
    for i, word in enumerate(cut_list):
        try:
            # 将词转换为索引index
            cut_list[i] = cn_model.vocab[word].index
        except KeyError:
            # 如果词不在字典中，则输出0
            cut_list[i] = 0
    train_tokens_positive.append(cut_list)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.588 seconds.
Prefix dict has been built succesfully.


In [13]:
train_tokens_negative = []
for text in negative_comment:
    text = re.sub("[\s+\.\!\/_$%^*(+]+|[+——！，。；：《》【】“”？、~@#￥%……&*（）]+", "",text)
    cut_text = jieba.cut(text)
    cut_list = [i for i in cut_text]
    for i, word in enumerate(cut_list):
        try:
            # 将词转换为索引index
            cut_list[i] = cn_model.vocab[word].index
        except KeyError:
            # 如果词不在字典中，则输出0
            cut_list[i] = 0
    train_tokens_negative.append(cut_list)

In [14]:
len(train_tokens_negative)

763

In [15]:
len(train_tokens_positive)

3812

#### 反向tokenize
- 定义一个方法，用来把索引转换成可阅读的文本。



In [16]:
# 用来将tokens转换为文本
def reverse_tokens(tokens):
    text = ''
    for i in tokens:
        if i != 0:
            text = text + cn_model.index2word[i]
        else:
            text = text + ' '
    return text

In [17]:
r1 = reverse_tokens(train_tokens_positive[0])
r1

'包贝尔演坏人真是本色演出最后被暴打确实让人解气你们都说翻拍老手但这部真的比老手强'

#### 准备Embedding Matrix
- 准备embedding matrix（词向量矩阵），维度为 (𝑛𝑢𝑚𝑤𝑜𝑟𝑑𝑠,𝑒𝑚𝑏𝑒𝑑𝑑𝑖𝑛𝑔𝑑𝑖𝑚)(numwords,embeddingdim) ，num words代表我们使用的词汇的数量，emdedding dimension在现在使用的预训练词向量模型中是300，每一个词汇都用一个长度为300的向量表示。
- 只选择使用前50000个使用频率最高的词，。

In [52]:
embedding_dim

300

In [21]:
# 只使用前50000个词
num_words = 50000
# 初始化embedding_matrix，之后在keras上进行应用
embedding_matrix = np.zeros((num_words, embedding_dim))
# embedding_matrix为一个 [num_words，embedding_dim] 的矩阵
# 维度为 50000 * 300
for i in range(num_words):
    embedding_matrix[i,:] = cn_model[cn_model.index2word[i]]
embedding_matrix = embedding_matrix.astype('float32')

In [22]:
# 检查index是否对应，
# 输出300意义为长度为300的embedding向量一一对应
np.sum( cn_model[cn_model.index2word[888]] == embedding_matrix[888] )

300

In [64]:
embedding_matrix.shape

(50000, 300)

In [25]:
# 获得所有positive tokens的长度
num_tokens_positive = np.array([ len(tokens) for tokens in train_tokens_positive ])
num_tokens_positive

array([25, 94, 45, ...,  5, 63,  5])

In [26]:
# 平均positive_tokens的长度
np.mean(num_tokens_positive)

44.792235047219307

In [27]:
# 获得所有negative tokens的长度
num_tokens_negative = [ len(tokens) for tokens in train_tokens_negative ]
num_tokens_negative = np.array(num_tokens_negative)
num_tokens_negative

array([106,  66, 174, 191,  78, 112,   6,  72,  53,  57,  13,  99,   9,
        53,  35,   2,  26,  85,  68,  18,  12,  55,  95,  73, 106,  66,
       174, 191,  78, 112,   6,  72,  53,  57,  13,  99,   9,  53,  35,
         2,  26,  85,  68,  18,  12,  55,  95,  73,  89,   6,  32,   5,
        21,  59,  13,  22,  16,  75,  24,  75,  12,  90,  18,  20,  44,
        24,  69,  15,  39,  25,   4,  17,  11,  46, 199,  41, 148,   4,
        74,  19,  45,  88,  35,   6,  67, 108,  83,  21,   8,  14,  11,
        15,  18,  10,  62,  15,  75,  22,  62,  68,  71,   6,  17,   5,
        65,  72,  13,  70,   7,  70,  37,  22,  70,  41,  58,  29,  72,
        68,  18,  40,  69,  76,  16,  11,  15,  64,  46,  62,  58,  47,
        13,  12,   8,  57,  11,  15, 188,   7,  65,  38, 186, 106,  80,
       153,  88,  49,  99,  97,  58,  22,  22,  24,   3,  13, 129,  17,
        10,  61,   1,  22,  37,  75,   2,  24,  63,  66,  28,  22,  33,
         5,  14,   7,  73,  30,  47,   3,  72,  10, 149,   4,  6

In [56]:
# 平均negative_tokens的长度
np.mean(num_tokens_negative)

47.807339449541281

In [28]:
# 取tokens平均值并加上两个tokens的标准差，
# 假设tokens长度的分布为正态分布，则max_tokens这个值可以涵盖95%左右的样本
max_tokens_positive = int(np.mean(num_tokens_positive) + 2 * np.std(num_tokens_positive))
max_tokens_positive

118

In [29]:
max_tokens_negative = int(np.mean(num_tokens_negative) + 2 * np.std(num_tokens_negative))
max_tokens_negative

134

In [31]:
max_tokens = 134

In [32]:
# 取tokens的长度为236时，大约95%的样本被涵盖
# 我们对长度不足的进行padding，超长的进行修剪
np.sum( num_tokens_negative < max_tokens ) / len(num_tokens_negative)

0.94233289646133678

#### padding（填充）和truncating（修剪）
- 把文本转换为tokens之后，每一串索引的长度并不相等，需要把索引的长度标准化，选择了134这个可以涵盖94.2%训练样本的长度，接下来进行padding和truncating，这会在文本索引的前面填充0.

In [33]:
train_tokens = train_tokens_positive + train_tokens_negative
len(train_tokens)

4575

In [34]:
# 进行padding和truncating， 输入的train_tokens是一个list
# 返回的train_pad是一个numpy array
train_pad = pad_sequences(train_tokens, maxlen=max_tokens,
                            padding='pre', truncating='pre')

In [35]:
# 超出五万个词向量的词用0代替
train_pad[ train_pad>=num_words ] = 0

In [36]:
# 可见padding之后前面的tokens全变成0，文本在最后面
train_pad[888]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0, 4461,  313, 7072,
       1067,    0], dtype=int32)

In [67]:
# 准备target向量
train_target = np.concatenate( (np.ones(len(train_tokens_positive)),
                                np.zeros(len(train_tokens_negative))) )
train_target

array([ 1.,  1.,  1., ...,  0.,  0.,  0.])

In [38]:
X_train, X_test, y_train, y_test = train_test_split(train_pad,
                                                    train_target,
                                                    test_size=0.1,
                                                    random_state=1)

In [39]:
display(reverse_tokens(X_train[66]))
display('class: ',y_train[66])

'                                                                                                                           编剧算是融入了一点带有思考的本土化给及格'

'class: '

1.0

- 用keras搭建LSTM模型，模型的第一层是Embedding层，只有把tokens索引转换为词向量矩阵之后，才可以用神经网络对文本进行处理。
- keras提供了Embedding接口，避免了繁琐的稀疏矩阵操作。   
- 在Embedding层输入的矩阵为：$$(batchsize, maxtokens)$$
- 输出矩阵为： $$(batchsize, maxtokens, embeddingdim)$$

In [40]:
# 搭建LSTM模型
model_LSTM = Sequential()

In [94]:
max_tokens

134

In [41]:
# 模型第一层为embedding
model_LSTM.add(Embedding(num_words,
                    embedding_dim,
                    weights=[embedding_matrix],
                    input_length=max_tokens,
                    trainable=False))

In [42]:
model_LSTM.add(Bidirectional(LSTM(units=32, return_sequences=True)))
model_LSTM.add(LSTM(units=16, return_sequences=False))

#### 构建模型
- BiLSTM：测试了LSTM和BiLSTM，发现BiLSTM的表现最好
- Embedding之后第一层我们用BiLSTM返回sequences，然后第二层16个单元的LSTM不返回sequences，只返回最终结果，最后是一个全链接层，用sigmoid激活函数输出结果。

In [43]:
model_LSTM.add(Dense(1, activation='sigmoid'))
model_LSTM.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

In [44]:
model_LSTM.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 134, 300)          15000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 134, 64)           85248     
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                5184      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 15,090,449
Trainable params: 90,449
Non-trainable params: 15,000,000
_________________________________________________________________


In [50]:
# 建立一个权重的存储点
path_checkpoint = 'sentiment_checkpoint.keras'
checkpoint = ModelCheckpoint(filepath=path_checkpoint, monitor='val_loss',
                                      verbose=1, save_weights_only=True,
                                      save_best_only=True)

In [51]:
# 尝试加载已训练模型
try:
    model_LSTM.load_weights(path_checkpoint)
except Exception as e:
    print(e)

Unable to open file (File read failed: time = fri jan 25 15:04:19 2019
, filename = 'sentiment_checkpoint.keras', file descriptor = 56, errno = 21, error message = 'is a directory', buf = 0x7ffe26f3a220, total read size = 8, bytes this sub-read = 8, bytes actually read = 18446744073709551615, offset = 0)


In [53]:
# 定义early stoping如果3个epoch内validation loss没有改善则停止训练
earlystopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [54]:
# 自动降低learning rate
lr_reduction = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1, min_lr=0.00001, patience=0,
                                       verbose=1)

In [58]:
# 定义callback函数
callbacks = [
    earlystopping, 
    lr_reduction
]

In [59]:
# 开始训练
model_LSTM.fit(X_train, y_train,
          validation_split=0.1, 
          epochs=20,
          batch_size=128,
          callbacks=callbacks)

Train on 3705 samples, validate on 412 samples
Epoch 1/20
3705/3705 [==============================] - 26s - loss: 0.4524 - acc: 0.8318 - val_loss: 0.4293 - val_acc: 0.8447
Epoch 2/20
3705/3705 [==============================] - 26s - loss: 0.4470 - acc: 0.8318 - val_loss: 0.4286 - val_acc: 0.8447
Epoch 3/20
3705/3705 [==============================] - 27s - loss: 0.4348 - acc: 0.8318 - val_loss: 0.3979 - val_acc: 0.8447
Epoch 4/20
3705/3705 [==============================] - 28s - loss: 0.3922 - acc: 0.8318 - val_loss: 0.3897 - val_acc: 0.8447
Epoch 5/20
3584/3705 [============================>.] - ETA: 0s - loss: 0.3697 - acc: 0.8362
Epoch 00004: reducing learning rate to 0.00010000000474974513.
3705/3705 [==============================] - 29s - loss: 0.3724 - acc: 0.8343 - val_loss: 0.3980 - val_acc: 0.8519
Epoch 6/20
3705/3705 [==============================] - 30s - loss: 0.3366 - acc: 0.8483 - val_loss: 0.3849 - val_acc: 0.8617
Epoch 7/20
3705/3705 [==============================

In [60]:
def predict_sentiment(text):
    print(text)
    # 去标点
    text = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",text)
    # 分词
    cut = jieba.cut(text)
    cut_list = [ i for i in cut ]
    # tokenize
    for i, word in enumerate(cut_list):
        try:
            cut_list[i] = cn_model.vocab[word].index
        except KeyError:
            cut_list[i] = 0
    # padding
    tokens_pad = pad_sequences([cut_list], maxlen=max_tokens,
                           padding='pre', truncating='pre')
    # 预测
    result = model_LSTM.predict(x=tokens_pad)
    coef = result[0][0]
    if coef >= 0.5:
        print('好评','output=%.2f'%coef)
    else:
        print('差评','output=%.2f'%coef)

In [66]:
predict_sentiment('太烂了！')

太烂了！
好评 output=0.57


In [65]:
predict_sentiment("差评")

差评
好评 output=0.58
